Module 9 part of Tensorflow Lite

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
model = keras.models.load_model('clothing-model.h5')

In [4]:
#download the picture
!wget http://bit.ly/mlbookcamp-pants -O pants.jpg

--2023-11-27 07:15:16--  http://bit.ly/mlbookcamp-pants
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg [following]
--2023-11-27 07:15:16--  https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23048 (23K) [image/jpeg]
Saving to: ‘pants.jpg’

pants.jpg           100%[===================>]  22.51K  --.-KB/s    in 0.001s  

2023-11-27 07:15:16 (15.8 MB/s) - ‘pants.jp

In [8]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np


In [7]:
img = load_img('pants.jpg', target_size=(299,299))

In [9]:
X = np.array(img)
X = np.array([X])

X = preprocess_input(X)

In [10]:
preds = model.predict(X)

1/1 [==============================] - 2s 2s/step


In [12]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -1.8798649,
 'hat': -4.7563114,
 'longsleeve': -2.359532,
 'outwear': -1.0892639,
 'pants': 9.903785,
 'shirt': -2.8261807,
 'shoes': -3.648312,
 'shorts': 3.2411556,
 'skirt': -2.612096,
 't-shirt': -4.8520355}

## Convert keras to TF-Lite

In [13]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('clothing-model.tflite', 'wb') as f_out:
  f_out.write(tflite_model)

In [14]:
import tensorflow.lite as tflite

In [15]:
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
interpreter.allocate_tensors()

In [17]:
interpreter.get_input_details()

[{'name': 'serving_default_input_8:0',
  'index': 0,
  'shape': array([  1, 299, 299,   3], dtype=int32),
  'shape_signature': array([ -1, 299, 299,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [18]:
input_index = interpreter.get_input_details()[0]['index']

In [19]:
#need to see the output too
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 229,
  'shape': array([ 1, 10], dtype=int32),
  'shape_signature': array([-1, 10], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [20]:
output_index = interpreter.get_output_details()[0]['index']

In [23]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [24]:
#now try to predict
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -1.8798652,
 'hat': -4.7563086,
 'longsleeve': -2.3595319,
 'outwear': -1.0892631,
 'pants': 9.903782,
 'shirt': -2.8261807,
 'shoes': -3.648309,
 'shorts': 3.2411587,
 'skirt': -2.6120942,
 't-shirt': -4.8520336}

## Removing TF Dependency

In [25]:
#to change the load_img (aka write from scratch)
#before: img = load_img('pants.jpg', target_size=(299,299))
#after:

from PIL import Image

with Image.open('pants.jpg') as img:
  img = img.resize((299, 299), Image.NEAREST)

In [26]:
#now write from scratch for preprocess_input()

def preprocess_input(x):
  x /= 127.5
  x -= 1.
  return x

In [29]:
X = np.array(img, dtype = 'float32')
X = np.array([X])

X = preprocess_input(X)

In [30]:
#now try to predict
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -1.8798652,
 'hat': -4.7563086,
 'longsleeve': -2.3595319,
 'outwear': -1.0892631,
 'pants': 9.903782,
 'shirt': -2.8261807,
 'shoes': -3.648309,
 'shorts': 3.2411587,
 'skirt': -2.6120942,
 't-shirt': -4.8520336}

## Simpler doing of removing tf depedency

In [31]:
!pip install keras-image-helper

In [34]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.6 MB/s eta 0:00:00


In [35]:

#import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [36]:
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [37]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [38]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [39]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [40]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -1.8798652,
 'hat': -4.7563086,
 'longsleeve': -2.3595319,
 'outwear': -1.0892631,
 'pants': 9.903782,
 'shirt': -2.8261807,
 'shoes': -3.648309,
 'shorts': 3.2411587,
 'skirt': -2.6120942,
 't-shirt': -4.8520336}